## Aufgabe: Crawler

**Aufgabe:**

- Passe den ArticleFetcher so an, dass er die Informationen aus allen Seiten extrahiert

Hier nochmal die URL: http://python.beispiel.programmierenlernen.io/index.php

**Tipps:**

- Schau dir zuerst an, wie du den Button "Zur nächsten Seite" ansteuern kannst.
- Wie greifst du von Python aus auf das "href" - Attribut dieses Buttons zu?
- (Optional): Probier ggf. zuerst, nur die Infos der ersten 2 Seiten zu holen. Kannst du darauf aufbauend das Programm verallgemeinern, so dass es alle Seiten einliest?
- Du kannst dich daran orientieren, ob es einen "Zur nächsten Seite"-Button gibt, oder nicht. Wenn es diesen Button nicht mehr gibt, bist du auf der letzten Seite angelangt. Welcher Schleifentyp bietet sich hier an, wenn du die Schleife erst dann stoppen willst, wenn es den Button nicht mehr gibt?

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

import time

In [3]:
class CrawledArticle():

    def __init__(self, title, emoji, content, image):
        self.title = title
        self.emoji = emoji
        self.content = content
        self.image = image

    def __str__(self):
        return self.emoji + ": " + self.title

    def to_csv(self):
        return [self.emoji, self.title, self.content, self.image]

    @staticmethod
    def csv_header():
        return ["Emoji", "Title", "Content", "ImageUrl"]

class NextPage():

    def __init__(self, url, links):
        self.has_next_page = False
        self.links = links
        for link in links:
             if link.text == "Zur nächsten Seite!":
                self.url_args  = {}
                for arg in link.attrs["href"].split('?'):
                    if '=' in arg:
                        (k, v) = arg.split("=")
                        self.url_args[k] = v
                self.has_next_page = "page" in self.url_args
        self.url = urljoin(url, f"?page={self.url_args['page']}") if self.has_next_page else None

    def __str__(self):
        return self.url

class ArticleFetcher():
    def __init__(self, base_url):
        self.url = base_url
    
    def fetch(self, limit_pages = 0, limit_entries = 0):
        url = self.url
        page_count = 0
        entry_count = 0

        while(url != None):
            if limit_pages > 0 and page_count >= limit_pages:
                break
            time.sleep(1)
            print("read data from " + url)
            r = requests.get(url)
            doc = BeautifulSoup(r.text, "html.parser")
            next_page = NextPage(url, doc.select("div.navigation a"))
            for card in doc.select(".card"):
                if limit_entries > 0 and entry_count >= limit_entries:
                    url = None
                    break
                emoji = card.select_one(".emoji").text
                content = card.select_one(".card-text").text
                title = card.select(".card-title span")[1].text
                image = urljoin(url, card.select_one("img").attrs["src"])
                yield CrawledArticle(title, emoji, content, image)
                entry_count += 1

            page_count += 1
            url = next_page.url

    



In [13]:
import csv

fetcher = ArticleFetcher("http://python.beispiel.programmierenlernen.io/index.php")

with open("data\crawler_output.csv", 'w', newline='', encoding='utf-8') as csv_file:

    article_writer = csv.writer(csv_file, delimiter = ';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    article_writer.writerow(CrawledArticle.csv_header())

    for article in fetcher.fetch():
        article_writer.writerow(article.to_csv())

read data from http://python.beispiel.programmierenlernen.io/index.php
read data from http://python.beispiel.programmierenlernen.io/index.php?page=2
read data from http://python.beispiel.programmierenlernen.io/index.php?page=3
read data from http://python.beispiel.programmierenlernen.io/index.php?page=4
read data from http://python.beispiel.programmierenlernen.io/index.php?page=5
read data from http://python.beispiel.programmierenlernen.io/index.php?page=6
